In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import copy
import seaborn as sns
from scipy.stats import t, ortho_group
from collections import defaultdict 
from tqdm import tqdm
from sklearn.linear_model import Lasso
sns.set(rc={'figure.figsize':(16,8)})

# EX 1

In [7]:
class Model:

  def __init__(self, k: int, gen_method: str):
    if gen_method == 'normal':
        self.X = np.random.normal(0, 1/np.sqrt(1000), size=(1000,950))
    elif gen_method == 'ortho':
        self.X = ortho_group.rvs(dim=1000)
        self.X = self.X[:,:950]

    self.beta = np.array([[3.5 for _ in range(k)] + [0 for i in range(950-k)]]).T
    self.eps = np.random.normal(0, 1, size=(1000, 1))
    self.n = self.X.shape[0]
    self.k = k
    self.p = 950
    self.n = self.X.shape[0]



  def solve_ols(self):
    Y = self.get_true_Y()
    self.beta_ols = np.linalg.inv(self.X.T @ self.X) @ self.X.T @ Y
    Y_pred = self.get_perdicted_Y('OLS')
    self.variances_ols = np.linalg.inv(self.X.T @ self.X) * 1/(self.n - self.p) * ((Y - Y_pred)**2).sum()
    self.variances_ols = self.variances_ols.diagonal().flatten()
    
  def solve_ridge(self):
    Y = self.get_true_Y()
    lamb = self.get_lambda()
    self.beta_ridge = np.linalg.inv(self.X.T @ self.X + np.eye(self.p)*lamb)@self.X.T@Y
    Y_pred = self.get_perdicted_Y('RIDGE')
    # self.variances_ridge = np.linalg.inv(self.X.T @ self.X) * 1/(self.n - self.p) * ((Y - Y_pred)**2).sum()
    self.variances_ridge = 1*np.linalg.inv(self.X.T @ self.X + np.eye(self.p)*lamb)@self.X.T@self.X@np.linalg.inv(self.X.T @ self.X + np.eye(self.p)*lamb)
    self.variances_ridge = self.variances_ridge.diagonal().flatten() 
    
  def get_bias_and_variance(self, method: str, i: int):
    if method == 'OLS':
        return self.beta_ols[i] - self.beta[i], self.variances_ols[i]
    if method == 'RIDGE':
        return self.beta_ridge[i] - self.beta[i], self.variances_ridge[i]
    if method == 'THEORETICAL':
        return (-1*self.p)/(np.linalg.norm(self.beta, ord=2)**2 + 1*self.p)*self.beta[i], \
                ((1*self.beta[i]**2)/(1*self.p + self.beta[i]**2))**2

  def get_mse(self, method: str):
    if method == 'THEORETICAL':
        return (np.linalg.norm(self.beta, ord=2)**2 * 1 * self.p) / (np.linalg.norm(self.beta, ord=2)**2 + 1*self.p)
    if method == 'OLS':
        return np.sum((self.beta - self.beta_ols)**2)
    if method == 'RIDGE':
        return np.sum((self.beta - self.beta_ridge)**2)

  def get_perdicted_Y(self, method):
    if method == 'OLS':
        return self.X @ self.beta_ols
    if method == 'RIDGE':
        return self.X @ self.beta_ridge
  
  def get_true_Y(self):
    return (self.X @ self.beta) + self.eps
  
  def get_lambda(self):
      return self.p/(np.linalg.norm(self.beta, ord=2)**2)


In [8]:
for k in [20, 100, 200]:
    print(f'k={k}')
    model = Model(k, 'ortho')
    print(f'lambda={model.get_lambda()}')
    model.solve_ridge()
    model.solve_ols()
    
    for method in ['THEORETICAL', 'RIDGE', 'OLS']:
        bias, var = model.get_bias_and_variance(method, 2)
        mse = model.get_mse(method)
        print(f'{method}: {bias}, {var}, {mse}')
    
    

k=20
lambda=3.8775510204081636
THEORETICAL: [-2.78242678], [0.00016207], 194.7698744769874
RIDGE: [-2.66458931], 0.04203357784352514, 196.23986449749813
OLS: [0.57475826], 1.1912161236322112, 906.6936985436841
k=100
lambda=0.7755102040816326
THEORETICAL: [-1.52873563], [0.00016207], 535.0574712643678
RIDGE: [-1.18776468], 0.3172149557405206, 546.5383048845005
OLS: [0.6053974], 0.7811906118256362, 950.5828380556563
k=200
lambda=0.3877551020408163
THEORETICAL: [-0.97794118], [0.00016207], 684.5588235294117
RIDGE: [-1.45232238], 0.5192474048442904, 637.6734191406413
OLS: [-0.65832494], 0.9994240712716915, 897.093817238474


# Ex 2

In [37]:
class Model:

  def __init__(self, k: int, gen_method: str):
    if gen_method == 'normal':
        self.X = np.random.normal(0, 1/np.sqrt(1000), size=(1000,950))
    elif gen_method == 'ortho':
        self.X = ortho_group.rvs(dim=1000)
        self.X = self.X[:,:950]

    self.beta = np.array([[3.5 for _ in range(k)] + [0 for i in range(950-k)]]).T
    self.eps = np.random.normal(0, 1, size=(1000, 1))
    self.n = self.X.shape[0]
    self.k = k
    self.p = 950
    self.n = self.X.shape[0]

    
  def solve_ridge(self, policy):
    Y = self.get_true_Y()
    lamb = self.get_lambda(policy)
    self.beta_ridge = np.linalg.inv(self.X.T @ self.X + np.eye(self.p)*lamb)@self.X.T@Y
    Y_pred = self.X @ self.beta_ridge
    # self.variances_ridge = np.linalg.inv(self.X.T @ self.X) * 1/(self.n - self.p) * ((Y - Y_pred)**2).sum()
    self.variances_ridge = 1*np.linalg.inv(self.X.T @ self.X + np.eye(self.p)*lamb)@self.X.T@self.X@np.linalg.inv(self.X.T @ self.X + np.eye(self.p)*lamb)
    self.variances_ridge = self.variances_ridge.diagonal().flatten() 

  def solve_lasso(self, policy, alpha):
    Y = self.get_true_Y()
    lam = self.get_lambda(policy)
    self.lasso_model = Lasso(alpha=alpha)
    self.lasso_model.fit(self.X, Y)

  def get_mse(self, method: str):
    if method == 'LASSO':
        return 0
    if method == 'RIDGE':
        return np.sum((self.beta - self.beta_ridge)**2), \
               np.sum((self.X@(self.beta - self.beta_ridge))**2)

  def get_true_Y(self):
    return (self.X @ self.beta) + self.eps
  
  def get_lambda(self, policy):
    if policy == 'min_PE':
        return 0
    elif policy == '10_CV':
        return 1
    elif policy == 'OLS':
        return 2
    elif policy == 'OLS_AIC':
        return 3
    return self.p/(np.linalg.norm(self.beta, ord=2)**2)

In [55]:
model = Model(100, 'normal')
model.solve_lasso('a', 0.005)

24


In [56]:
np.sum(model.lasso_model.coef_ > 0)
model.lasso_model.coef_

array([ 0.        ,  0.        ,  0.40009679,  0.        ,  0.        ,
        0.        ,  0.00866386,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.63062482,  0.        ,  0.44925451,
        0.        ,  0.        ,  0.        ,  1.33632934,  0.        ,
        1.7296418 ,  0.        ,  0.8157773 ,  0.18249082,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.59696657,  1.31409867,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.37703952,  0.        ,
        0.31987154,  0.        ,  0.357531  ,  0.        ,  1.33816569,
        0.4995621 ,  0.        ,  0.94363532,  0.        ,  0.77797435,
        0.        ,  0.        ,  0.67661003,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.28

In [ ]:
for k in [20, 100, 200]:
    print(f'k={k}')
    model = Model(k, 'norm')
    
    
    for policy in ['min_PE', '10_CV', 'OLS', 'OLS_AIC']:
        model.solve_ridge(policy)
        model.solve_lasso(policy)
        mse_lasso, mse_ridge = model.get_mse('LASSO'), model.get_mse('RIDGE')
        print(f'{policy}: [beta_mse]: {mse_lasso[0]}, {mse_ridge[0]}\n
                [eps_mse]: {mse_lasso[1]}, {mse_ridge[1]}')